In [ ]:
#install package
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
#import file for training
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
#save file as dataframe
import io
#df = pd.read_csv(io.BytesIO(uploaded['train.csv']))
df = pd.read_csv('train.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
#preprocessing
df.drop(['keyword', 'location', 'id'], axis=1, inplace=True)
stop = stopwords.words('english')
np.random.seed(19970901)

def url(text):
    stuff = re.compile(r'https?://\S+|www\.\S+')
    return stuff.sub(r'', text)

def emoji(text):
    stuff = re.compile(
        '['
        u'\U0001F600-\U0001F64F'
        u'\U0001F300-\U0001F5FF'
        u'\U0001F680-\U0001F6FF'
        u'\U0001F1E0-\U0001F1FF'
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return stuff.sub(r'', text)

def html(text):
    stuff = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(stuff, '', text)

def punctuation(text):
    stuff = str.maketrans('', '', string.punctuation)
    return text.translate(stuff)

df['text'] = df['text'].apply(lambda x: url(x))
df['text'] = df['text'].apply(lambda x: emoji(x))
df['text'] = df['text'].apply(lambda x: html(x))
df['text'] = df['text'].apply(lambda x: punctuation(x))
df['text'] = df['text'].apply(lambda x: str.lower(x))
#df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#split into train and test
train = df.iloc[:6090, :]
test = df.iloc[6091:, :]

In [ ]:
#specify simple transformer model with number of epochs
model_args = ClassificationArgs(num_train_epochs = 5,learning_rate = 3e-5)
model = ClassificationModel(
    "roberta", "roberta-base", args=model_args, use_cuda=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
# If this is not the first time running this, remove eval_results.txt from output directory. Otherwise, no need to run this chunk.
%ls
%cd outputs/
%ls
%rm eval_results.txt
%ls

In [ ]:
#Train model and evaluate test set
model.train_model(train)
results, model_outputs, predictions = model.eval_model(test)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/762 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# Calculate F1-score
tp = results.get('tp')
tn = results.get('tn')
fp = results.get('fp')
fn = results.get('fn')
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * (precision*recall)/(precision + recall)
print(f1)